In [98]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select

import os
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
#
# Seleniumをあらゆる環境で起動させるオプション
#
options = Options()
options.add_argument('--disable-gpu');
options.add_argument('--disable-extensions');
options.add_argument('--proxy-server="direct://"');
options.add_argument('--proxy-bypass-list=*');
options.add_argument('--start-maximized');

#環境情報色々
DRIVER_PATH = '/Users/hayashiryuutarou/Desktop/study/webcrow/selenium/chromedriver'

#各種Keyの読み込み
import configparser
#config取得
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')
ACCOUNT = config_ini["SBI"]["ACCOUNT"]
PASSWORD = config_ini["SBI"]["PASSWORD"]
TRANSACTION_PASSWORD = config_ini["SBI"]["TRANSACTION_PASSWORD"]
name = config_ini["SBI"]["name"]

In [2]:
#ログイン関数
def connect_sbi(ACCOUNT, PASSWORD, name,driver):
    # SBI証券のトップ画面を開く
    driver.get('https://www.sbisec.co.jp/ETGate')
    
    # ユーザーIDとパスワード
    input_user_id = driver.find_element_by_name('user_id')
    input_user_id.send_keys(ACCOUNT)
    input_user_password = driver.find_element_by_name('user_password')
    input_user_password.send_keys(PASSWORD)
    
    # ログインボタンをクリック
    driver.find_element_by_name('ACT_login').click()
    time.sleep(5)

    return driver

In [3]:
def start():
    option = Options()
    option.add_argument('--headless') 
    #driverの設定
    driver = webdriver.Chrome(executable_path=DRIVER_PATH, chrome_options=options)
    #サイトログイン
    driver = connect_sbi(ACCOUNT, PASSWORD, name,driver)
    return driver

In [4]:
driver = start()

/Users/hayashiryuutarou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [105]:
def transact(driver,params):
    driver.find_element_by_xpath('//div[@id="link02M"]/ul/li[2]').click()
    #取引タイプ選択
    driver.find_element_by_id(params["取引"]).click()
    #銘柄コード
    driver.find_element_by_name("stock_sec_code").send_keys(params["銘柄コード"])
    #市場選択は不要か？
    
    #株数
    driver.find_element_by_name("input_quantity").send_keys(params["株数"])
    #取引方法
    transact_elem = driver.find_element_by_id(params["取引方法"])
    transact_elem.find_element_by_name("in_sasinari_kbn").click()
    #取引追加情報(価格とか)
    if params["取引方法"] == "gsn0": #指値
        #条件
        select_elem = Select(transact_elem.find_element_by_name("sasine_condition"))
        select_elem.select_by_value(params["取引詳細"]["条件"])
        #価格
        transact_elem.find_element_by_name("input_price").send_keys(params["取引詳細"]["価格"])
    if params["取引方法"] == "gsn1": #成行
        #条件
        select_elem = Select(transact_elem.find_element_by_name("nariyuki_condition"))
        select_elem.select_by_value(params["取引詳細"]["条件"])
    #期間
    #預かり区分
    driver.find_element_by_name("hitokutei_trade_kbn").click()
    #信用取引区分
    
    #取引パスワードを入力
    driver.find_element_by_name("trade_pwd").send_keys(TRANSACTION_PASSWORD)
    #注文確認画面をスキップ
    driver.find_element_by_name("skip_estimate").click()
    #注文発注
    driver.find_element_by_id("botton2").click()

In [106]:
#取引詳細のサンプル
#指値
transact_prams = {
    "条件":" ",
    "価格":150
}

params={
    "取引":"genU",  #現物買:genK,現物売:genU,信用新規買:shinK,信用新規売:shinU
    "銘柄コード":1400,
    "株数":100,
    "取引方法":"gsn0", #指値：gsn0,成行:gsn1,逆指値:gsn2 ※逆指値は未実装
    "取引詳細":transact_prams,
    "期間":1,
    "期間詳細":"20/05/27",
    "預かり区分":"hitokutei_trade_kbn", #一般預かり：hitokutei_trade_kbn,特定預かり:
    "信用取引区分":"制度"
}
transact(driver,params)